## Deliverable 2. Create a Customer Travel Destinations Map.

In [53]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [54]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,69.4541,-133.0374,50.00,81,100,10.36,Overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,78.53,55,40,9.22,Scattered clouds
2,2,Baykit,RU,61.6700,96.3700,51.69,93,84,2.30,Broken clouds
3,3,Busselton,AU,-33.6500,115.3333,47.79,85,0,9.62,Clear sky
4,4,Lewistown,US,40.5992,-77.5714,86.34,65,40,6.91,Scattered clouds


In [55]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you desire for your trip?  "))
max_temp = float(input("What is the maximum temperature you desire for your trip?  "))

In [56]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)] 

preferred_cities_df.head(10)
len(preferred_cities_df)

138

In [57]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [58]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. (just in case a random data set pulls some with nas)
clean_df = preferred_cities_df.dropna()

In [59]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuktoyaktuk,CA,50.00,Overcast clouds,69.4541,-133.0374,
2,Baykit,RU,51.69,Broken clouds,61.6700,96.3700,
3,Busselton,AU,47.79,Clear sky,-33.6500,115.3333,
5,Chokurdakh,RU,51.93,Few clouds,70.6333,147.9167,
7,Barrow,US,51.82,Light rain,71.2906,-156.7887,
21,Hobart,AU,48.06,Broken clouds,-42.8794,147.3294,
27,Yar-Sale,RU,43.65,Overcast clouds,66.8333,70.8333,
29,Ushuaia,AR,37.02,Scattered clouds,-54.8000,-68.3000,
32,Narsaq,GL,46.62,Overcast clouds,60.9167,-46.0500,
38,Kayerkan,RU,53.53,Overcast clouds,69.3497,87.7602,


In [60]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000, # this is defined in meters by default
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Vacation Error 404: Hotel not found!")
        

Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!
Vacation Error 404: Hotel not found!


In [61]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuktoyaktuk,CA,50.00,Overcast clouds,69.4541,-133.0374,End of The Road Ltd.
3,Busselton,AU,47.79,Clear sky,-33.6500,115.3333,Observatory Guest House
7,Barrow,US,51.82,Light rain,71.2906,-156.7887,King Eider Inn
21,Hobart,AU,48.06,Broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
27,Yar-Sale,RU,43.65,Overcast clouds,66.8333,70.8333,Yalemd
29,Ushuaia,AR,37.02,Scattered clouds,-54.8000,-68.3000,Albatross Hotel
32,Narsaq,GL,46.62,Overcast clouds,60.9167,-46.0500,Sleep In Narsaq
39,Kurilsk,RU,47.19,Overcast clouds,45.2333,147.8833,Hotel Ostrov
41,Bluff,NZ,45.81,Overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
44,Mmabatho,ZA,49.51,Clear sky,-25.8500,25.6333,Mahikeng Guest Apartments


In [62]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
len(clean_hotel_df)

109

In [66]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Weather Description</b></dt><dd>{Current Description}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [67]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
figure_layout = {
    'height': '1500px'
}

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0, layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='1500px'))